# **Crypto assets market price and users behavior on Compound**
-  The main purpose of this notebook is to present a data exploratory visualization and see if there are some explicit correlation between market price of crypto assets & user behavior (with utilization_ratio as proxy)
-	Use Granger causality to test the lead/lag autocorrelation between two variables. Don't forget to check the model's assumption (e.g., stationary).
    - https://www.youtube.com/watch?v=XqsSB_vpHLs (Granger Causality)
    - https://www.youtube.com/watch?v=5jOW6baXYI4 (5 ways to fail with time series)
-	Check if the utilization ratio is correlated with the liquidity mining activities (use cETH to borrow ETH). Maybe we should find a way to remove self-borrowing from calculating the utilization ratio. (cf. daily liability matrix)


### Packages init

In [1]:
import altair as alt
import pandas as pd
import numpy as np

# Load data

market_price = pd.read_csv('daily_markets.csv') # Load market price data
full_market_price = market_price
market_price = market_price[market_price['underlyingSymbol'].isin(['WBTC', 'ETH'])] # Filter out non-ETH/WBTC markets
print(market_price.columns) # Print columns
alt.data_transformers.enable('csv') # Enable csv data transformer


Index(['borrowRate', 'cash', 'collateralFactor', 'exchangeRate',
       'interestRateModelAddress', 'name', 'reserves', 'supplyRate', 'symbol',
       'cTokenAddress', 'totalBorrows', 'totalSupply', 'underlyingAddress',
       'underlyingName', 'underlyingPrice', 'underlyingSymbol',
       'accrualBlockNumber', 'blockTimestamp', 'borrowIndex', 'reserveFactor',
       'underlyingPriceUSD', 'underlyingDecimals', 'date', 'newPrice'],
      dtype='object')


DataTransformerRegistry.enable('csv')

### Plotting Data

### Price

In [ ]:
# Plot market prices double y-axis
prices = alt.Chart(market_price).mark_line().encode(
    x='date:T',
    y='underlyingPriceUSD:Q',
    color='underlyingSymbol:N',
)

prices

## Interest Rate

See https://altair-viz.github.io/gallery/select_detail.html to enhance rendering of plot

In [ ]:
# Plot market borrow rates
interest_rate = alt.Chart(market_price).mark_line().encode(
    x='date:T',
    y='borrowRate:Q',
    color='underlyingSymbol:N',
)


## Utilization Ratio

In [ ]:
market_price['utilization_ratio'] = market_price['totalBorrows'] / ( market_price['totalBorrows'] + market_price['cash'] - market_price['reserves'] )
print(market_price.columns)

u_ratio = alt.Chart(market_price).mark_line().encode(
    x='date:T',
    y='utilization_ratio:Q',
    color='underlyingSymbol:N',
)



u_ratio | interest_rate | prices

## Log Return

In [ ]:
import datetime
# Log return of crypto market prices grouped by underlyingSymbol

market_price['yield'] = market_price.groupby('underlyingSymbol')['underlyingPriceUSD'].pct_change()


# plot yield

yield_chart = alt.Chart(market_price).mark_line().encode(
    x='date:T',
    y='yield:Q',
    color='underlyingSymbol:N',
)

print(market_price.columns)
# Display
 
prices | yield_chart | u_ratio



In [ ]:
#overlap WBTC yield and WBTC utilization ratio

wbtc = market_price[market_price['underlyingSymbol'] == 'WBTC']

wbtc_yield = alt.Chart(wbtc).mark_line(opacity=1, color='red').encode(
    x='date:T',
    y='yield:Q',

)

wbtc_utilization_ratio = alt.Chart(wbtc).mark_line(opacity=1, color='green').encode(
    x='date:T',
    y='utilization_ratio:Q',
)

wbtc_price = alt.Chart(wbtc).mark_line(interpolate="monotone").encode(
    x='date:T',
    y='underlyingPriceUSD:Q',
)


wbtc_yield & wbtc_utilization_ratio

In [ ]:
wbtc_price & wbtc_utilization_ratio

In [ ]:
wbtc_yield.mark_bar() & wbtc_price

## Max Yield

In [ ]:
alt.Chart(wbtc).mark_rect().encode(
    alt.X("date(date):O").axis(labelAngle=0, format="%e").title("Day"),
    alt.Y("month(date):O").title("Month"),
    alt.Color("max(yield):Q").title("Yield"),
)

## Max Utilization Ratio

In [ ]:
alt.Chart(wbtc).mark_rect().encode(
    alt.X("date(date):O").axis(labelAngle=0, format="%e").title("Day"),
    alt.Y("month(date):O").title("Month"),
    alt.Color("max(utilization_ratio):Q").title("Utilization Ratio"),
)

## Max Price

In [ ]:
alt.Chart(wbtc).mark_rect().encode(
    alt.X("date(date):O").axis(labelAngle=0, format="%e").title("Day"),
    alt.Y("month(date):O").title("Month"),
    alt.Color("max(underlyingPriceUSD):Q").title("Price"),
)

In [ ]:

# reindex market price with grouping by underlyingSymbol and sympol

wbtc.head()

## **Investigating on potential correlation between coins market price and utilization_ratio**
- Time series analysis
    - Granger Causality
    - Time Lagged Cross Correlation
    - Dynamic Time Warping
    - Instantaneous phase synchrony
    - Fourier Transform
    - Power Spectrum Distribution
- Focus on Granger Causality
    - Computing the GC = log(V(e1)/V(e2)) on a **time-window** *(parameter s)* and the **model order** *(parameter k)* of the Autoregressive Model
        - Short **time-window** => worse model estimation but better temporal resolution
        - Long **time-window** => better model estimation but worse temporal resolution
        - Lower **order** => better model estimation (faster compuationt time) but less sensitive to history
        - Higher **order** => worse model estimation (longer compuationt time) but more sensitive to history
    - The MVGC multivariate Granger causality toolbox: a new approach to Granger-causal inference
        -The standard method of computing G-causality involves estimation of parameters for both a full and a nested (reduced) VAR model. The MVGC approach, by contrast, avoids explicit estimation of the reduced model, thus eliminating a source of estimation error and improving statistical power, and in addition facilitates fast and accurate estimation of the computationally awkward case of conditional G-causality in the frequency domain.
    - Trying to make GC test on differents time periods (2019-2020 / 2020-2021 / 2021-2022 / 2022-2023)

In [ ]:
eth_utilization_ratio = alt.Chart(market_price[market_price['underlyingSymbol'] == 'ETH']).mark_line(opacity=1, color='green').encode( x='date:T', y='utilization_ratio:Q').properties(title=alt.Title(text='ETH Utilization Ratio'))
wtbc_utilization_ratio = alt.Chart(market_price[market_price['underlyingSymbol'] == 'WBTC']).mark_line(opacity=1, color='green').encode( x='date:T', y='utilization_ratio:Q').properties(title=alt.Title(text='WBTC Utilization Ratio'))
eth_price = alt.Chart(market_price[market_price['underlyingSymbol'] == 'ETH']).mark_line(interpolate="monotone").encode( x='date:T', y='underlyingPriceUSD:Q').properties(title=alt.Title(text='ETH Price'))
wbtc_price = alt.Chart(market_price[market_price['underlyingSymbol'] == 'WBTC']).mark_line(interpolate="monotone").encode( x='date:T', y='underlyingPriceUSD:Q').properties(title=alt.Title(text='WBTC Price'))
eth_yield = alt.Chart(market_price[market_price['underlyingSymbol'] == 'ETH']).mark_line(opacity=1, color='red').encode( x='date:T', y='yield:Q').properties(title=alt.Title(text='ETH Yield'))
(eth_utilization_ratio | wbtc_utilization_ratio) & ( eth_yield | wbtc_yield)

 - Let's check granger causality between coin market price (wtbc or eth) and utilization ratio
 - https://youtu.be/4TkNZviNJC0 (Granger causality tests in python)

## Example of stationnary time serie, adfuller test and granger causality test

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests, adfuller

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
#build the time series, just a simple AR(1)
t1 = [0.1*np.random.normal()]
for _ in range(100):
    t1.append(0.5*t1[-1] + 0.1*np.random.normal())

In [ ]:
#build the time series that is granger caused by t1
t2 = [item + 0.1*np.random.normal() for item in t1]
# adding a little bit of noise to t1 to make it more realistic

In [ ]:
#adjust t1 and t2 creating a 3 day lag in t1
t1 = t1[3:]
t2 = t2[:-3]

In [ ]:
# testing adf for t1
adf1 = adfuller(t1)
adf1

In [ ]:
# testing adf for t2
adf2 = adfuller(t2)
adf2

In [ ]:
#plotting t1 and t2

plt.figure(figsize=(10,4))
plt.plot(t1, color='b')
plt.plot(t2, color='r')

plt.legend(['t1', 't2'], fontsize=16)

In [ ]:

ts_df = pd.DataFrame(columns=['t2', 't1'], data=zip(t2,t1))
ts_df.head()


In [ ]:
#testing if t2 granger causes t1 up to 5 lags

gc_results = grangercausalitytests(ts_df, 5)

- Clearly we see that reaching a GC 3-lag test leads us with p-value = 0.0 implying that 3 is the good order to explain t1 from t2

### Testing GC between market_price of wtbc and ETH and their respective utilization_ratio **without data cleaning** in a first time

In [ ]:

# granger causality test for wbtc underlyingPriceUSD on utilization_ratio

wbtc_ts_df = pd.DataFrame(columns=['utilization_ratio','underlyingPriceUSD'], data=zip(wbtc['utilization_ratio'],wbtc['underlyingPriceUSD']))
wbtc_ts_df.head()

In [ ]:
# adfuller test for wbtc underlyingPriceUSD
wbtc_adf1 = adfuller(wbtc_ts_df['underlyingPriceUSD'])
wbtc_adf1

In [ ]:
#testing if underlyingPriceUSD granger causes utilization_ratio up to 5 lags
gc_results = grangercausalitytests(wbtc_ts_df, 5, addconst=True)


- Really bad result : the best p-value is 0.69 for a 1-lag assumption and increase to 1 with the order test.
- Let's do the same with eth price and its utilization ratio

In [ ]:

# filtering for eth only

eth = market_price[market_price['underlyingSymbol'] == 'ETH']

eth.head()


In [ ]:
# granger causality test for eth underlyingPriceUSD on utilization_ratio

eth_ts_df = pd.DataFrame(columns=['utilization_ratio','underlyingPriceUSD'], data=zip(eth['utilization_ratio'],eth['underlyingPriceUSD']))
eth_ts_df.columns


In [ ]:
# adfuller test for eth underlyingPriceUSD
eth_adf1 = adfuller(eth_ts_df['underlyingPriceUSD'])
eth_adf1

In [ ]:

#testing if underlyingPriceUSD granger causes utilization_ratio up to 5 lags
gc_results = grangercausalitytests(eth_ts_df, 5)

- better results than WBTC GC test but still high for the p-value = 0.15 for 2 lag
### Let's try again  **with data cleaning** this time
- https://youtu.be/7_Js8h709Dw (Cleaning data for time-series)

In [ ]:
# select row 54 of eth_ts_df
eth_ts_df.loc[54]


In [ ]:
# normalize with pandas the data for eth and wbtc

eth_ts_df = (eth_ts_df - eth_ts_df.mean()) / eth_ts_df.std()
wbtc_ts_df = (wbtc_ts_df - wbtc_ts_df.mean()) / wbtc_ts_df.std()


# removing trend from the data (first property of a stationary time series)

eth_ts_df = eth_ts_df.diff().dropna()
wbtc_ts_df = wbtc_ts_df.diff().dropna()

# normalizing by a periodic, lets say 1 year, volatility (second property of a stationary time series)
"""
for i in range(0, len(eth_ts_df),365):
    eth_ts_df[i:i+365] /= eth_ts_df[i:i+365].std()

for i in range(0, len(wbtc_ts_df),365):
    wbtc_ts_df[i:i+365] /= wbtc_ts_df[i:i+365].std()
"""
#plotting eth_price and wbt_price

plt.figure(figsize=(10,4))
plt.plot(eth_ts_df['underlyingPriceUSD'], color='b')
plt.plot(wbtc_ts_df['underlyingPriceUSD'], color='r')

plt.legend(['eth_price', 'wbtc_price'], fontsize=16)
plt.title('Cleaned ETH and WBTC Price', fontsize=20)

#plotting eth_utilization_ratio and wbtc_utilization_ratio

plt.figure(figsize=(10,4))
plt.plot(eth_ts_df['utilization_ratio'], color='b')
plt.plot(wbtc_ts_df['utilization_ratio'], color='r')

plt.legend(['eth_utilization_ratio', 'wbtc_utilization_ratio'], fontsize=16)
plt.title('Cleaned ETH and WBTC Utilization Ratio', fontsize=20)


In [ ]:
#adfuller test for wbtc underlyingPriceUSD
wbtc_adf1 = adfuller(wbtc_ts_df['underlyingPriceUSD'])
wbtc_adf1


In [ ]:
gc_results = grangercausalitytests(wbtc_ts_df, 10)

In [ ]:
#adfuller test for eth underlyingPriceUSD
eth_adf1 = adfuller(wbtc_ts_df['underlyingPriceUSD'])
eth_adf1

In [ ]:
gc_results = grangercausalitytests(eth_ts_df, 5)

In [ ]:
#testing stationnarity of utilization_ratio for eth and wbtc

eth_adf2 = adfuller(eth_ts_df['utilization_ratio'])
wbtc_adf2 = adfuller(wbtc_ts_df['utilization_ratio'])

print("ETH URATIO",eth_adf2)
print("WBTC URATIO", wbtc_adf2)

## Trying grangercausality with price changes (daily return) instead of market price

In [ ]:
# granger causality test for eth yield on utilization_ratio

eth_ts2_df = pd.DataFrame(columns=['yield','utilization_ratio'], data=zip(eth['yield'],eth['utilization_ratio'])).dropna()
eth_ts2_df.head()

# granger causality test for wbtc yield on utilization_ratio

wbtc_ts2_df = pd.DataFrame(columns=['utilization_ratio','yield'], data=zip(wbtc['utilization_ratio'],wbtc['yield'])).dropna()
wbtc_ts2_df.head()



In [ ]:
# normalize with pandas the data for eth and wbtc

eth_ts2_df = (eth_ts2_df - eth_ts2_df.mean()) / eth_ts2_df.std()
wbtc_ts2_df = (wbtc_ts2_df - wbtc_ts2_df.mean()) / wbtc_ts2_df.std()


# removing trend from the data (first property of a stationary time series)

eth_ts2_df = eth_ts2_df.diff().dropna()
wbtc_ts2_df = wbtc_ts2_df.diff().dropna()

In [ ]:

# adfuller test for eth yield

eth_adf3 = adfuller(eth_ts2_df['yield'])
eth_adf3


In [ ]:

#testing if yield granger causes utilization_ratio up to 5 lags
gc_results = grangercausalitytests(eth_ts2_df, 5)

In [ ]:
#plotting eth_utilization_ratio and eth_yield

plt.figure(figsize=(10,4))
plt.plot(eth_ts2_df['yield'], color='r')
plt.plot(eth_ts2_df['utilization_ratio'], color='b')


plt.legend(['eth_utilization_ratio', 'eth_yield'], fontsize=16)
plt.title('Cleaned ETH Utilization Ratio and Yield', fontsize=20)


In [ ]:
# adfuller test for wbtc yield

wbtc_adf3 = adfuller(wbtc_ts2_df['yield'])
wbtc_adf3



In [ ]:
# granger causality test for wbtc yield on utilization_ratio on 10 lags
gc_results = grangercausalitytests(wbtc_ts2_df, 10)

#plotting p_value of granger causality test for wbtc yield in function of lags

p_values = [gc_results[i+1][0]['ssr_ftest'][1] for i in range(10)]
plt.figure(figsize=(10,4))
plt.plot(p_values, color='b')
plt.title('P-value of Granger Causality Test for WBTC Yield', fontsize=20)
plt.xlabel('Lags')
plt.ylabel('P-value')

# function that return the best p-value for a granger causality test and its regarding lag

def best_p_value(gc_results):
    p_values = [gc_results[i+1][0]['ssr_ftest'][1] for i in range(10)]
    return min(p_values), p_values.index(min(p_values)) + 1

print(best_p_value(gc_results))


In [ ]:
#plotting wbtc_utilization_ratio and wbtc_yield

plt.figure(figsize=(10,4))
plt.plot(wbtc_ts2_df['yield'], color='r')
plt.plot(wbtc_ts2_df['utilization_ratio'], color='b')


plt.legend(['wbtc_utilization_ratio', 'wbtc_yield'], fontsize=16)
plt.title('Cleaned WBTC Utilization Ratio and Yield', fontsize=20)




## Next Step
<input type="checkbox"> Dickey-Fuller Test to quantify the degree of stationarity </input>

<input type="checkbox"> Improve stationarity cleaning (seasonal normalization) </input>

<input type="checkbox" checked> Is u-ratio very stationnary </input>

<input type="checkbox"> Trying grangercausality with price changes (daily return) instead of market price </input>

<input type="checkbox"> cETH, cDAI and cUSDT and utilization ratio </input>

<input type="checkbox"> grangercausality in the other side (t2,t1) </input>

<input type="checkbox"> generalizing the process of gc test on others crypto-assets and see if we can classify differents families of crypto-assets (wbtc like or eth like) </input>


In [ ]:
# Function that takes : data, a max number of lag, a symbol, two columns (t1,t2) 
# and return the best p-value and its regarding lag for a granger causality test after cleaning the data

def Granger(data, max_lag, symbol, t1, t2, plotting=False):
    
    # filtering for symbol only
    data = data[data['underlyingSymbol'] == symbol]
    
    # build the time series
    ts_df = pd.DataFrame(columns=[t1,t2], data=zip(data[t1],data[t2])).dropna()
    
    #if the data is not stationary, make it stationary
    if adfuller(ts_df[t1])[1] > 0.05:
        ts_df[t1] = ( ts_df[t1] - ts_df[t1].mean() ) / ts_df[t1].std()
        ts_df[t1] = ts_df[t1].diff().dropna()
    if adfuller(ts_df[t2])[1] > 0.05:
        ts_df[t2] = ( ts_df[t2] - ts_df[t2].mean() ) / ts_df[t2].std()
        ts_df[t2] = ts_df[t2].diff().dropna()
    ts_df = ts_df.dropna()
    
    # testing if t2 granger causes t1 up to max_lag
    gc_results = grangercausalitytests(ts_df, max_lag,verbose=False)
    
    # return the best p-value and its regarding lag
    p_values = [gc_results[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]
    
    #plotting p_value of granger causality test for t2 on t1 in function of lags
    if plotting:
        plt.figure(figsize=(10,4))
        plt.plot(p_values, color='b')
        plt.title('P-value of Granger Causality Test for '+symbol+' '+t2+' on '+t1, fontsize=20)
        plt.xlabel('Lags')
        plt.ylabel('P-value')

    return min(p_values), p_values.index(min(p_values)) + 1


In [ ]:
full_market_price['yield'] = full_market_price.groupby('underlyingSymbol')['underlyingPriceUSD'].pct_change(fill_method=None)
full_market_price['utilization_ratio'] = full_market_price['totalBorrows'] / ( full_market_price['totalBorrows'] + full_market_price['cash'] - full_market_price['reserves'] )
symbols = full_market_price['underlyingSymbol'].unique()
full_market_price.columns

In [ ]:
for symbol in symbols:
    if symbol in ["USDC","TUSD","USDP","FEI"]:
        continue
    print(symbol)
    # print(Granger(full_market_price, 50, symbol, 'utilization_ratio', 'yield', True))


In [ ]:
# create a dataframe with the Granger function for each symbol and for each pair of columns among ['underlyingPriceUSD', 'utilization_ratio', 'yield']

granger_list = []
for symbol in symbols:
    for t1 in ['underlyingPriceUSD', 'utilization_ratio', 'yield', 'totalSupply', 'collateralFactor']:
        for t2 in ['underlyingPriceUSD', 'utilization_ratio', 'yield', 'totalSupply', 'collateralFactor']:
            if t1 != t2:
                try:
                    p_value, lag = Granger(full_market_price, 10, symbol, t1, t2)
                    granger_list.append({'symbol': symbol, 't1': t1, 't2': t2, 'p_value': p_value, 'lag': lag})
                except:
                    print("Error with ", symbol, t1, t2)

granger_dfs = pd.DataFrame(granger_list)

granger_dfs.head()

In [ ]:

# Vizualizing the p-value of the granger causality test for each pair of columns and for each symbol with altair make darker the p-value is low

alt.Chart(granger_dfs).mark_rect().encode(
    x='t1:N',
    y='t2:N',
    color=alt.Color('p_value:Q', scale=alt.Scale(scheme='darkblue')),
    column='symbol:N'
).properties(
    width=200,
    height=200
)


In [ ]:
# ganger causality test between WBTC prices and ETH

def GrangerCrossSymbol(data, max_lag, symbol1, symbol2, t1, t2, plotting=False):
        
    # filtering for symbol only
    data1 = data[data['underlyingSymbol'] == symbol1]
    data2 = data[data['underlyingSymbol'] == symbol2]
    
    # build the time series
    c1 = symbol1+'_'+t1
    c2 = symbol2+'_'+t2
    ts_df = pd.DataFrame(columns=[c1,c2], data=zip(data1[t1],data2[t2])).dropna()
    
    #if the data is not stationary, make it stationary
    if adfuller(ts_df[c1])[1] > 0.05:
        ts_df[c1] = ( ts_df[c1] - ts_df[c1].mean() ) / ts_df[c1].std()
        ts_df[c1] = ts_df[c1].diff().dropna()
    if adfuller(ts_df[c2])[1] > 0.05:
        ts_df[c2] = ( ts_df[c2] - ts_df[c2].mean() ) / ts_df[c2].std()
        ts_df[c2] = ts_df[c2].diff().dropna()
    ts_df = ts_df.dropna()
    
    # testing if t2 granger causes t1 up to max_lag
    gc_results = grangercausalitytests(ts_df, max_lag,verbose=False)
    
    # return the best p-value and its regarding lag
    p_values = [gc_results[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]
    
    #plotting p_value of granger causality test for t2 on t1 in function of lags
    if plotting:
        plt.figure(figsize=(10,4))
        plt.plot(p_values, color='b')
        plt.title('P-value of Granger Causality Test for '+symbol2+' '+t2+' on '+symbol1+' '+t1, fontsize=20)
        plt.xlabel('Lags')
        plt.ylabel('P-value')

    return min(p_values), p_values.index(min(p_values)) + 1
    

In [ ]:
GrangerCrossSymbol(full_market_price, 20, 'WBTC', 'ETH', 'underlyingPriceUSD', 'underlyingPriceUSD', True)

In [ ]:
def Granger_p_values(data, max_lag, symbol1, symbol2, t1, t2):
        
    # filtering for symbol only
    data1 = data[data['underlyingSymbol'] == symbol1]
    data2 = data[data['underlyingSymbol'] == symbol2]
    
    # build the time series
    c1 = symbol1+'_'+t1
    c2 = symbol2+'_'+t2
    ts_df = pd.DataFrame(columns=[c1,c2], data=zip(data1[t1],data2[t2])).dropna()
    
    #if the data is not stationary, make it stationary
    if adfuller(ts_df[c1])[1] > 0.05:
        ts_df[c1] = ( ts_df[c1] - ts_df[c1].mean() ) / ts_df[c1].std()
        ts_df[c1] = ts_df[c1].diff().dropna()
    if adfuller(ts_df[c2])[1] > 0.05:
        ts_df[c2] = ( ts_df[c2] - ts_df[c2].mean() ) / ts_df[c2].std()
        ts_df[c2] = ts_df[c2].diff().dropna()
    ts_df = ts_df.dropna()
    
    # testing if t2 granger causes t1 up to max_lag
    gc_results = grangercausalitytests(ts_df, max_lag,verbose=False)
    
    # return the best p-value and its regarding lag
    p_values = [gc_results[i+1][0]['ssr_ftest'][1] for i in range(max_lag)]

    return p_values

In [ ]:
# Function that takes : data, a max number of lag, a list of symbol, two columns (t1,t2) 
# and return a dataframe with granger causality test p-value for all the lags from 0 to max_lag between t1 and t2 for each symbol after cleaning the data
# then plot with alatair vega the dataframe with different color for each symbol on a same graph

def GrangerSymbols(data, max_lag, symbols, t1, t2, plotting=False):
    
    granger_list = []
    for token in symbols: 
        try:
            p_values = Granger_p_values(data, max_lag, token, token, t1, t2)
            for i in range(len(p_values)):
                granger_list.append({'symbol': token, 'lag': i, 'p_value': p_values[i]})
        except:
            print("Error with ", token, t1, t2)
        
    
    granger_dfs = pd.DataFrame(granger_list)

    if plotting:
        chart = alt.Chart(granger_dfs).mark_line().encode(
            x='lag',
            y='p_value:Q',
            color='symbol:N'
        ).properties(
            title='Granger Causality Test: ' + t2 + ' causing ' + t1
        )

        rule = alt.Chart(pd.DataFrame({'y': [0.05]})).mark_rule(color='red', strokeDash=[5, 5]).encode(y='y')
        chart += rule
        
        return chart
    
    return granger_dfs

In [ ]:
valid_symbols = ['REP', 'ETH', 'BAT', 'ZRX', 'DAI', 'WBTC', 'USDT', 'UNI', 'COMP', 'LINK', 'SUSHI', 'YFI', 'AAVE']
my_symbols = ['ETH', 'WBTC', 'USDT', 'COMP', 'DAI']
GrangerSymbols(full_market_price, 50, valid_symbols , 'totalSupply', 'underlyingPriceUSD', True)

- [x] Focus study on : SC (Stable coin) (DAI  USDC USDT) / VC (Volatile Coin) (WTBC  ETH  COMP)
- [x] Scaling the plot of p-value in absolute axis (for the y axis)
- [ ] Trying to classify the different patterns
- [x] Some interpretation and hypothesis of utilization ratio correlation with other columns
- [x] See Defi Leverage paper [link](https://www.bis.org/publ/work1171.htm)
- [ ] Plot historical events on charts with https://bitcoin.fr/Histoire/ or http://help.cryptosheets.com/en/articles/2262493-how-to-pull-events-and-calendar-data-for-cryptocurrencies-into-excel


In [5]:
# print range of date of market_price
print(market_price['date'].min(), "->" , market_price['date'].max())

2019-05-07 -> 2023-12-31


In [2]:
# chart of wbtc and eth price

eth_price = alt.Chart(market_price[market_price['underlyingSymbol'] == 'ETH']).mark_line(interpolate="monotone").encode( x='date:T', y='underlyingPriceUSD:Q').properties(title=alt.Title(text='ETH Price'))
wbtc_price = alt.Chart(market_price[market_price['underlyingSymbol'] == 'WBTC']).mark_line(interpolate="monotone").encode( x='date:T', y='underlyingPriceUSD:Q').properties(title=alt.Title(text='WBTC Price'))

eth_price | wbtc_price

alt.HConcatChart(...)

In [6]:
import requests
token = "your_token"

# make a request to coindar API v2 that Returns a cryptocurrencies list
coins_response = requests.get("https://coindar.org/api/v2/coins?access_token={token}")

# get the data from the response
coins_dump = coins_response.json()
coins_id = { data["symbol"]:data["id"] for data in coins_dump }


In [ ]:

# make a request to coindar API v2 that Returns a list of events tags
tags_response = requests.get("https://coindar.org/api/v2/tags?access_token={token}")

# get the data from the response
tags_dump = tags_response.json()
tags_id = { data["name"]:data["id"] for data in tags_dump }


In [ ]:

# make a request to coindar API v2 that Returns cryptocurrency events

"""
Parameters :

access_token (required) — access token.

page — page number.
By default: 1

page_size — page size.
By default: 30
Possible values: from 1 to 100

filter_date_start — the lower limit of the event start date in the format yyyy-mm-dd.
By default: date of the first event on Coindar.

filter_date_end — the upper date limit of the event start date in the format yyyy-mm-dd.
By default: date of the last event on Coindar by the time of request.

filter_coins — cryptocurrencies IDs, relating to the requested events.
By default: all cryptocurrencies
Example: 1,2,3,4,10

filter_tags — tags IDs (list of tags IDs can be received by the tags method).
By default: all tags
Example: 1,2,3

sort_by — parameter, defining events sorting.
By default: date_start
Possible values: date_start (start date), date_added (publication date), views (amount of views). Remember, that views accounting started in June 2018.

order_by — sorting order
By default: 0
Possible values: 0 (in the increasing order) or 1 (in the decreasing order)

Example : 
https://coindar.org/api/v2/events?access_token={token}&page=1&page_size=30&filter_date_start=2018-06-01&filter_date_end=2018-07-01&filter_coins=1,4,20,34&filter_tags=1,2,3&sort_by=views&order_by=1

Response example :
[
{
"caption": "Listing on OTCBTC",
"source": "https://coindar.org/en/event/oraclechain-oct-listing-on-otcbtc-11299",
"source_reliable": "true",
"important": "false",
"date_public": "2018-7-09 17:23",
"date_start": "2018-7-11 06:00",
"date_end": "",
"coin_id": "280",
"coin_price_changes": "-12.89",
"tags": "9"
}, 
... ]

"""

page = 1
page_size = 30
filter_date_start = "2018-06-01"
filter_date_end = "2018-07-01"
filter_coins = "1,4,20,34"
filter_tags = "1,2,3"
sort_by = "views"
order_by = "1"

event_response = requests.get(f"https://coindar.org/api/v2/events?access_token={token}&page={page}&page_size={page_size}&filter_date_start={filter_date_start}&filter_date_end={filter_date_end}&filter_coins={filter_coins}&filter_tags={filter_tags}&sort_by={sort_by}&order_by={order_by}")

# get the data from the response
event_dump = event_response.json()
